# Введение в обработку текста на естественном языке

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Введение в обработку текста на естественном языке"
* https://www.nltk.org/api/nltk.metrics.distance.html
* https://pymorphy2.readthedocs.io/en/stable/user/guide.html
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Задачи для совместного разбора

In [4]:
!pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 44.9 MB/s eta 0:00:00:00:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=1c0a9370aeca7cb470e562927d94b687b23566353b6f94e34bb73eb42eb9f59d
  Stored in directory: /home/codespace/.cache/pip/wheels/7c/d7/8d/2156234738063e3d4a39ba77dc677046100e62766b53807189
Successfully built docopt


In [58]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

1. Считайте слова из файла `litw-win.txt` и запишите их в список `words`. При помощи расстояния Левенштейна иправьте опечатку в слове "велечайшим".

In [ ]:
fil

2. Разбейте текст из формулировки второго задания на слова. Проведите стемминг и лемматизацию слов.

In [5]:
from nltk.stem import SnowballStemmer
from nltk import word_tokenize
import pymorphy2

In [ ]:
text = '''Разбейте текст из формулировки второго задания на слова. Проведите стемминг и лемматизацию слов.'''

3. Преобразуйте предложения из формулировки задания 2 в векторы при помощи `CountVectorizer`. Выведите на экран словарь обученного токенизатора.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk import sent_tokenize

In [ ]:
text = '''Разбейте текст из формулировки второго задания на слова. Проведите стемминг и лемматизацию слов.'''

## Лабораторная работа 7

1\. Загрузите данные из файла `ru_recipes_sample.csv` в виде `pd.DataFrame` `recipes` Используя регулярные выражения, удалите из описаний (столбец `description`) все символы, кроме русских букв, цифр и пробелов. Приведите все слова в описании к нижнему регистру. Сохраните полученный результат в столбец `description`.

In [166]:
import pandas as pd
import re

recipes = pd.read_csv("07_nlp_data/ru_recipes_sample.csv")

#recipes["description"] = recipes["description"].apply(lambda x: x if re.search(r"\w\w", x) else None)
recipes["description"] = recipes["description"].replace(to_replace=r"[^А-Яа-яЁё0-9\s]", value="", regex=True)
recipes["description"] = recipes["description"].apply(lambda x: x.lower())

recipes["description"]

0       этот коктейль готовлю из замороженной клубники...
1                                         быстро и вкусно
2                сытный овощной салатик пальчики оближете
3       картофельное пюре и куриные котлеты  вкусная к...
4       вишневая наливка имеет яркий вишневый вкус кот...
                              ...                        
3462    для тех кто любит чечевицу вам сюда очень вкус...
3463    баклажановые фантазии продолжаются предлагаю в...
3464    мое любимое блюдо лазанья но кушать только фар...
3465    прошлым летом варила варенье из одуванчиков по...
3466     и три корочки хлеба  сделал заказ буратино в ...
Name: description, Length: 3467, dtype: object

### Расстояние редактирования

2\. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние Левенштейна. Выведите на экран результат в следующем виде:

```
d(word1, word2) = x
```

In [167]:
unique_words = list(set(nltk.word_tokenize(" ".join(recipes['description']).replace('\r', ' ').replace('\n', ' '))))

unique_words[:10]

['шокоманы',
 'ночное',
 'резать',
 'овсяную',
 'акцент',
 'мнением',
 'виртуально',
 'бумагу',
 'несладкое',
 'поблагодарим']

In [168]:
import random

random_words = random.sample(unique_words, k=10)

for i in range(0, len(random_words) - 1, 2):
    print(f"d({random_words[i]}, {random_words[i+1]}) = {nltk.edit_distance(random_words[i], random_words[i+1])}")

d(каштаны, православный) = 8
d(милой, запекания) = 9
d(говяжьей, заверить) = 7
d(девушкам, решетке) = 6
d(вермишель, утончен) = 8


3\. Напишите функцию, которая принимает на вход 2 текстовые строки `s1` и `s2` и при помощи расстояния Левенштейна определяет, является ли строка `s2` плагиатом `s1`. Функция должна реализовывать следующую логику: для каждого слова `w1` из `s1` проверяет, есть в `s2` хотя бы одно слово `w2`, такое, что расстояние Левенштейна между `w1` и `w2` меньше 2, и считает количество таких слов в `s1` $P$. 

$$ P = \#\{w_1 \in s_1\ | \exists w_2 \in s_2 : d(w_1, w_2) < tol\}$$

$$ L = max(|s1|, |s2|) $$

Здесь $|\cdot|$ - количество слов в строке, $\#A$ - число элементов в множестве $A$, $w \in s$ означает, что слово $w$ содержится в тексте $s$.

Если отношение $P / L$ больше 0.8, то функция должна вернуть True; иначе False.

Продемонстрируйте работу вашей функции на примере описаний двух рецептов с ID 135488 и 851934 (ID рецепта - это число, стоящее в конце url рецепта). Выведите на экран описания этих рецептов и результат работы функции.

In [169]:
def is_plagiarism(s1: str, s2: str) -> bool:
    s1 = word_tokenize(s1)
    s2 = word_tokenize(s2)
    L = max(len(s1), len(s2))
    P = 0
    for word in s1:
        for word_2 in s2:
            if nltk.edit_distance(word, word_2) < 2:
                P += 1
    # print(P / L)
    return (P / L) > 0.8



In [170]:
# Рецепты с нужным нам ID

recipes[(recipes["url"].str.endswith("851934/")) | (recipes["url"].str.endswith("135488/"))]

,url,name,ingredients,description
958,https://www.povarenok.ru/recipes/show/135488/,Паштет сало-авокадо в хлебных орешках,"{'Сало': '100 г', 'Соль': '1/3 ч. л.', 'Чеснок...",прекрасной закуской к крепким напиткам на фурш...
1473,https://www.povarenok.ru/recipes/show/851934/,Паштет из сала и авокадов хлебных орешках,"{'Сало': '100 г', 'Соль': '1/3 ч. л.', 'Чеснок...",замечательной закуской к напиткам на фуршетном...


In [171]:
is_plagiarism(recipes[recipes["url"].str.endswith("135488/")]["description"].values[0], recipes[recipes["url"].str.endswith("851934/")]["description"].values[0])

True

### Стемминг, лемматизация

4\. На основе набора слов из задания 2 создайте `pd.DataFrame` со столбцами `word`, `stemmed_word` и `normalized_word`. В столбец `stemmed_word` поместите версию слова после проведения процедуры стемминга; в столбец `normalized_word` поместите версию слова после проведения процедуры лемматизации. Столбец `word` укажите в качестве индекса. 

Для стемминга можно воспользоваться `SnowballStemmer` из `nltk`, для лемматизации слов - пакетом `pymorphy2`. Сравните результаты стемминга и лемматизации. Поясните на примере одной из строк получившегося фрейма (в виде текстового комментария), в чем разница между двумя этими подходами. 

In [172]:
morph = pymorphy2.MorphAnalyzer()
stemmer = SnowballStemmer("russian")
stem_lem_df = pd.DataFrame(columns=['word', 'stemmed_word', 'normalized_word'])
stem_lem_df['word'] = unique_words
stem_lem_df['stemmed_word'] = [stemmer.stem(word) for word in stem_lem_df['word']]
stem_lem_df['normalized_word'] = [morph.parse(word)[0].normalized.word for word in stem_lem_df['word']]
stem_lem_df = stem_lem_df.set_index('word')
stem_lem_df

,stemmed_word,normalized_word
word,,
шокоманы,шокома,шокоман
ночное,ночн,ночной
резать,реза,резать
овсяную,овсян,овсяный
акцент,акцент,акцент
...,...,...
выбрать,выбра,выбрать
дичи,дич,дичь
готовлюсь,готовл,готовиться


5\. Добавьте в таблицу `recipes` столбец `description_no_stopwords`, в котором содержится текст описания рецепта после удаления из него стоп-слов. Посчитайте и выведите на экран долю стоп-слов среди общего количества слов. Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [159]:
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [173]:
stops = stopwords.words('russian')
pattern = r'\b(?:{})\b'.format('|'.join(stops))
recipes['description_no_stopwords'] = recipes['description'].str.replace(pattern, '')

/tmp/ipykernel_7395/1450924871.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  recipes['description_no_stopwords'] = recipes['description'].str.replace(pattern, '')


In [174]:
from collections import Counter


word_counts_before = Counter(word_tokenize(' '.join(recipes['description'])))
count_before, count_stops = sum(word_counts_before.values()), sum([word_counts_before[x] for x in stops])
count_before, count_stops, count_stops / count_before

(102978, 33223, 0.322622307677368)

In [175]:
word_counts_after = Counter(word_tokenize(' '.join(recipes['description_no_stopwords'])))
word_count_before = word_counts_before.most_common(10)
word_count_after = word_counts_after.most_common(10)
sum(word_counts_after.values()), count_before - count_stops

(69755, 69755)

In [176]:
word_count_before

[('и', 5054),
 ('в', 2584),
 ('с', 1934),
 ('на', 1655),
 ('очень', 1607),
 ('не', 1517),
 ('из', 1006),
 ('я', 979),
 ('рецепт', 869),
 ('а', 863)]

In [177]:
word_count_after

[('очень', 1607),
 ('рецепт', 869),
 ('это', 734),
 ('блюдо', 524),
 ('вкусный', 461),
 ('просто', 436),
 ('вкусно', 375),
 ('приготовить', 344),
 ('вкус', 324),
 ('салат', 313)]

### Векторное представление текста

6\. Выберите случайным образом 5 рецептов из набора данных, в названии которых есть слово "оладьи" (без учета регистра). Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`. На основе полученных векторов создайте `pd.DataFrame`, в котором названия колонок соответствуют словам из словаря объекта-векторизатора. 

Примечание: обратите внимание на порядок слов при создании колонок.

In [201]:
from sklearn.feature_extraction.text import TfidfVectorizer

olad = recipes[recipes['name'].str.contains('оладьи')]
sample = olad.sample(n=5)
sample

def get_dataframe_vect(sent_words):
    cv = TfidfVectorizer()
    cv.fit(sent_words)
    return pd.DataFrame([{elem: cv.transform(sent_words).toarray()[0][cv.vocabulary_[elem]] for elem in cv.vocabulary_}])

get_dataframe_vect(nltk.sent_tokenize(sample['description'].values[0]))

,нежные,сочные,оладьи,очень,вкусные
0,0.27735,0.27735,0.27735,0.83205,0.27735


7\. Вычислите близость между каждой парой рецептов, выбранных в задании 6, используя косинусное расстояние (можно воспользоваться функциями из любого пакета: `scipy`, `scikit-learn` или реализовать функцию самому). Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

Примечание: обратите внимание, что $d_{cosine}(x, x) = 0$

8\. Напишите функцию, которая принимает на вход `pd.DataFrame`, полученный в задании 7, и возвращает в виде кортежа названия двух различных рецептов, которые являются наиболее похожими. Прокомментируйте результат (в виде текстового комментария). Для объяснения результата сравните слова в описаниях двух этих отзывов.

In [ ]:
def find_closest(sim_df: pd.DataFrame) -> tuple:
    pass

In [15]:

from sklearn.feature_extraction.text import CountVectorizer
corpus = ['This is the first document.',
          'This document is second document.']
cv = CountVectorizer().fit(corpus)
print(cv.transform(corpus).toarray())

print(cv.get_feature_names())
# аrrаy([[1, 1, 1, 0, 1, 1],
#        [2, 0, 1, 1, 1, 1]], dtypе=int64)

[[1 1 1 0 1 1]
 [2 0 1 1 0 1]]
['document', 'first', 'is', 'second', 'the', 'this']
